# SQL

## Conexión a PostgreSQL usando SQLAlchemy

In [2]:
import os
import pandas as pd
from typing import Union
from sqlalchemy import create_engine, text

# Reemplaza USER y PASSWORD del .env
engine = create_engine("sqlite:///data/sql-murder-mystery.db")

# Probar la conexión
try:
    connection = engine.connect()
    print("Conexión exitosa")
    connection.close()
except Exception as e:
    print(f"Error al conectar: {e}")


def ejecutar_query(query: str) -> Union[pd.DataFrame, str]:
    """
    Ejecuta una consulta SQL en la base de datos y devuelve un DataFrame.

    Parámetros:
    - query (str): La consulta SQL a ejecutar.

    Retorna:
    - Union[pd.DataFrame, pl.DataFrame, str]: Retorna un DataFrame en pandas o polars dependiendo del parámetro 'devolver_polars',
      o un mensaje de éxito/error si la operación no es un SELECT.
    """
    
    with engine.connect() as connection:
        try:
            result = connection.execute(text(query))
            
            # Determinar si es una consulta SELECT para devolver datos
            if result.returns_rows:
                rows = result.fetchall()
                columns = result.keys()

                # Convertir filas en un diccionario de listas para crear correctamente el DataFrame en polars
                data_dict = {col: [row[idx] for row in rows] for idx, col in enumerate(columns)}

                # Verificación de coincidencia de columnas y filas
                return pd.DataFrame(data_dict)
            else:
                connection.commit()
                return f"Query ejecutada correctamente: {result.rowcount} filas afectadas."
        except Exception as e:
            return f"Error al ejecutar la query: {e}"


Conexión exitosa


In [5]:
query = 'SELECT * FROM crime_scene_report WHERE date = 20180115 AND city = "SQL City" AND type="murder" '

ejecutar_query(query=query)

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [6]:
query ='SELECT * FROM person WHERE address_street_name = "Northwestern Dr"  ORDER BY address_number DESC LIMIT 1'


ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [9]:
query = "SELECT * FROM person WHERE name LIKE '%Annabel%'AND address_street_name = 'Franklin Ave'"

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [10]:
query= 'SELECT * FROM interview WHERE person_id IN ("14887", "16371")'

ejecutar_query(query=query)

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


In [11]:
query= 'SELECT * FROM get_fit_now_check_in WHERE membership_id LIKE "48Z%" AND check_in_date = "20180109"'

ejecutar_query(query=query)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [12]:
query= 'SELECT * FROM drivers_license WHERE gender = "male" AND plate_number LIKE "%H42W%"'

ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [13]:
query= 'SELECT * FROM person WHERE license_id IN ("423327", "664760")'

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [15]:
query= "SELECT * FROM get_fit_now_member WHERE person_id IN ('51739', '67318')"

ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
